# Sample of 10,000 books on GoodReads.com

This notebook is a supplement to the analysis on the top 400 books on goodreads.com. The notebook for this analysis is called BestReads.ipynb.

We apply the same functions and code to a different list. No narrative in this notebook though. The full story is in the BestReads.ipynb notebook.

# Importing the modules and files

In [2]:
import csv
import os
import re
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# import matplotlib.pyplot as plt
# %matplotlib inline  
# from  matplotlib import style
# from matplotlib import rcParams

from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
from bokeh.plotting import figure
from bokeh.charts import *
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.charts.attributes import ColorAttr, CatAttr

Loading BokehJS ...

We will be doing a lot of plots in bokeh so we'll **create a function with the aesthetics we will use for each plot**.

In [3]:
def plot_aes(p, legend = "top_right"):
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.xaxis.minor_tick_line_color = None
    p.yaxis.minor_tick_line_color = None
    p.title.text_font_size = '16pt'
    p.outline_line_color = None
    p.legend.location = legend

Reading csv file & looking at five first observations:

In [4]:
goodreads = pd.read_csv('goodreads.csv')
goodreads.head()

,NumberOfPages,Genres,Description,BookCoverURL,Author,Reviews,MainGenre,Score,NumberOfReviews,NumberOfRating,Year,Title,AllGenres
0,157,Christian Fiction,Trying to heal the ache she feels in her empty...,https://www.goodreads.com/book/photo/32793383-...,Jody Hedlund,Available for Free on Kindle right now! Pure H...,Christian,4.29,37,146,2017,An Awakened Heart,"Christian Fiction,Romance,Historical,Novella,C..."
1,157,Christian Fiction,Trying to heal the ache she feels in her empty...,https://www.goodreads.com/book/photo/32793383-...,Jody Hedlund,Available for Free on Kindle right now! Pure H...,Christian,4.29,37,146,2017,An Awakened Heart,"Christian Fiction,Romance,Historical,Novella,C..."
2,157,Christian Fiction,Trying to heal the ache she feels in her empty...,https://www.goodreads.com/book/photo/32793383-...,Jody Hedlund,Available for Free on Kindle right now! Pure H...,Christian,4.29,37,146,2017,An Awakened Heart,"Christian Fiction,Romance,Historical,Novella,C..."
3,320,Nonfiction,"In the 1920s, the richest people per capita in...",https://www.goodreads.com/book/photo/29496076-...,David Grann,The Osage Indians lived in Kansas until the 18...,Crime,4.30,221,997,2017,Killers of the Flower Moon,"Nonfiction,History,Crime,True Crime,Mystery,Cr..."
4,157,Christian Fiction,Trying to heal the ache she feels in her empty...,https://www.goodreads.com/book/photo/32793383-...,Jody Hedlund,Available for Free on Kindle right now! Pure H...,Christian,4.29,37,146,2017,An Awakened Heart,"Christian Fiction,Romance,Historical,Novella,C..."


# Cleaning the data

First, let's make sure all of our observations are **unique**.

In [5]:
goodreads.drop_duplicates(inplace = True)

The columns 'Ranking', 'NumberOfPages', 'NumberOfReviews', 'NumberOfRatings' and 'Score' are already of numeric type.
We need to **turn the column 'TotalScore' into a numeric type**.

In our analysis, we will differentiate the books by their score. We'll create categories for the score and add a column **ScoreCat**.

In [6]:
goodreads['ScoreCat'] = goodreads['Score'].apply(lambda x: int(x)) # convert to integer

def scorecat(row):
    if (row['Score'] >= 0) and (row['Score'] <=1.5):
        return '1 star'
    elif (row['Score'] > 1.5) and (row['Score'] <=2.5):
        return '2 stars'
    elif (row['Score'] > 2.5) and (row['Score'] <=3.5):
        return '3 stars'
    elif (row['Score'] > 3.5) and (row['Score'] <=4.5):
        return '4 stars'
    else:
        return '5 stars'

goodreads['ScoreCat'] = goodreads.apply(lambda x: scorecat(x), axis = 1)

## The 'Reviews' column

The 'Reviews' column requires a little bit more cleaning. We will have to do some **stemming and lemmatization**. First we'll remove any puntuation and replace multiple white spaces with one. We'll take care of the stemming and lemmatization a bit later, when the time comes.

In [7]:
goodreads['Reviews'] = [re.sub( '[!"#$%&()*+\',-.:;<=>?@[]^_`{|}~/&]*', " ", reviews).strip() for reviews in goodreads['Reviews']]
goodreads['Reviews'] = [re.sub( '\s+', " ", reviews).strip() for reviews in goodreads['Reviews']]
goodreads['Reviews'] = [re.sub( '\d+', " ", reviews).strip() for reviews in goodreads['Reviews']]
goodreads['Reviews'] = [reviews.lower() for reviews in goodreads['Reviews']]

Removing the words that don't matter in the Reviews columns (e.g. I, and, the, this, etc.):

In [8]:
from nltk.corpus import stopwords
from nltk.tag import pos_tag

stop_words = set(stopwords.words("english"))
goodreads['Reviews'] = goodreads['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

We also remove any words with 1 or 2 letters:

In [9]:
shortword = re.compile(r'\W*\b\w{1,2}\b')
goodreads['Reviews'] = [shortword.sub('', word) for word in goodreads['Reviews']]

In [65]:
# save df as a csv to be able to use it in other notebook
goodreads.to_csv('files/section1b_number_of_pages.csv', index=False)

## 1- Writing


In [10]:
df_genre = goodreads.copy()
df_genre.AllGenres = df_genre.AllGenres.apply(lambda x: x.split(','))

In [11]:
df_genre.iloc[0].to_dict()
df_genre = pd.concat([pd.DataFrame(df_genre.iloc[i].to_dict()) for i in range(len(df_genre))])

In [12]:
df_genre.head()

,AllGenres,Author,BookCoverURL,Description,Genres,MainGenre,NumberOfPages,NumberOfRating,NumberOfReviews,Reviews,Score,ScoreCat,Title,Year
0,Christian Fiction,Jody Hedlund,https://www.goodreads.com/book/photo/32793383-...,Trying to heal the ache she feels in her empty...,Christian Fiction,Christian,157,146,37,available free kindle right now! pure hedlund ...,4.29,4 stars,An Awakened Heart,2017
1,Romance,Jody Hedlund,https://www.goodreads.com/book/photo/32793383-...,Trying to heal the ache she feels in her empty...,Christian Fiction,Christian,157,146,37,available free kindle right now! pure hedlund ...,4.29,4 stars,An Awakened Heart,2017
2,Historical,Jody Hedlund,https://www.goodreads.com/book/photo/32793383-...,Trying to heal the ache she feels in her empty...,Christian Fiction,Christian,157,146,37,available free kindle right now! pure hedlund ...,4.29,4 stars,An Awakened Heart,2017
3,Novella,Jody Hedlund,https://www.goodreads.com/book/photo/32793383-...,Trying to heal the ache she feels in her empty...,Christian Fiction,Christian,157,146,37,available free kindle right now! pure hedlund ...,4.29,4 stars,An Awakened Heart,2017
4,Christian,Jody Hedlund,https://www.goodreads.com/book/photo/32793383-...,Trying to heal the ache she feels in her empty...,Christian Fiction,Christian,157,146,37,available free kindle right now! pure hedlund ...,4.29,4 stars,An Awakened Heart,2017


In [13]:
len(set(df_genre['AllGenres'])) #there are 336 genres. 

336

In [14]:
from collections import Counter

# count number of books per genre
counter = Counter(df_genre['AllGenres']) # create dictionary with list of each genre and # of books per genre
counter_df = pd.DataFrame(counter.items(), columns = ['Genre', 'Count']) # turn dictionary into dataframe
counter_df = counter_df.sort_values('Count', ascending = False)

# replace name of Genre to "Other" when Genre has less than 75 books associated with it
counter_df['Genre'][counter_df['Count']<75] = 'Other'

counter_df = counter_df.groupby('Genre').sum()
counter_df_index = counter_df.reset_index()
counter_df_no_other = counter_df_index[counter_df_index['Genre'] != 'Other']

# convert count to %
counter_df_percent = counter_df.apply(lambda x: (x/sum(counter_df['Count'])*100))
counter_df_percent = counter_df_percent.reset_index()
counter_df_percent_no_other = counter_df_percent[counter_df_percent['Genre'] != 'Other']

# exlude others from analysis
counter_df_percent_no_other = counter_df_percent_no_other.sort_values(by = 'Count', ascending = False)

In [15]:
# plot the results (top 9 only)

counter_df_percent_no_other_top9 = counter_df_percent_no_other[0:9]

hover = HoverTool(
        tooltips=[
            ("Genre", "@Genre"),
            ('Percent','$y{1.1}%')
        ]
    )

palette_key = 3
p = Bar(counter_df_percent_no_other_top9,
        label=CatAttr(columns=['Genre'], sort=False), 
        values='Count',
        color = color(columns = 'Genre', palette = ['#d3ffce', '#ffd1d1', '#c6e2ff', '#dba087', '#d8be7e', '#b1d0ca','#cfb7ae', '#cfaec6', '#9ebbc6']),
        xlabel="",
        ylabel=" ",
        title="Percentage of Books by Genre",
        tools = [hover],
        plot_width=950, plot_height=600,
        legend = False
       )
# p.toolbar_location = None

plot_aes(p)
show(p)

In [17]:
# save df as a csv to be able to use it in other notebook
counter_df_percent_no_other.to_csv('files/section1a_books_per_genre.csv', index=False)

In [18]:
# create another dataframe where we group the genres and the ranking category
df_genre_stacked = df_genre[['ScoreCat','AllGenres']].groupby(['ScoreCat', 'AllGenres']).size().rename('Count')
df_genre_stacked = df_genre_stacked.reset_index()

# sort the columns by # of books in descending order
df_genre_stacked = df_genre_stacked.sort_values(by=['ScoreCat', 'Count'], ascending = False)

In [39]:
# plot the results
filtered_genres = ['Fiction','Fantasy','Classics','Romance','Young Adult', 'Literature','Historical','Science Fiction','Contemporary','Childrens','Cultural','Academic','Novels','Thirller','European Literature','Adventure','Audiobook']
df_genre_filtered = df_genre_stacked[df_genre_stacked['AllGenres'].isin(filtered_genres)]

hover = HoverTool(
        tooltips=[
            ("Genre", "$x"),
            ('Count', "@Count")
        ]
    )

data = {'Ranking Category': df_genre_filtered['ScoreCat'],
        'Count': df_genre_filtered['Count'],
        'Genre': df_genre_filtered['AllGenres']}

p = HeatMap(data, y='Ranking Category', x='Genre', values='Count',
             title='Stars by Category', stat=None, xlabel="",
            ylabel="",
            tools = [hover],
            plot_width=950, plot_height=600,palette=['#7ea4b3', '#9ebbc6', '#cedde2', '#bed1d8','#dee8eb']) 

plot_aes(p)
p.yaxis.major_label_orientation = 'horizontal'

show(p)

### Is there an optimal number of pages?

The second question revolves around the optimal number of pages. Let's look at the distribution of the top 400 books by number of pages.

In [40]:
hover = HoverTool(
        tooltips=[
            ("Number of pages", "$y{int}"),
            ('Score','$x')
        ]
    )

p = Scatter(goodreads, 
            x="Score", 
            y="NumberOfPages", 
            title="Distribution of Pages by Score",
            xlabel='Score', 
            ylabel='Number of Pages',
            tools = [hover, 'box_zoom', 'reset'],
            color = ['#7ea4b3'],
            plot_width=950, plot_height=400)
    
plot_aes(p)
show(p)

## 2- Publishing

### Does the year of publication matter?

In [41]:
# create df with year and # of books per year
year_count = goodreads[['NumberOfPages','Year']].groupby('Year').size().rename('Count').reset_index()
year_count

# create df with year and avg # of pages per year
year_pages = goodreads[['NumberOfPages','Year']].groupby('Year').mean().reset_index()
year_pages['NumberOfPages'] = year_pages['NumberOfPages'].round()

In [42]:
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span, Label

hover = HoverTool(
        tooltips=[
            ('Year','$x{int}'),
        ]
    )

citation = Label(x=2005, y=42,
                 text= '2006', 
                 render_mode='css', 
                 border_line_alpha=1.0, 
                 text_font_size = '10pt', 
                 text_color = 'black')

p = figure(plot_width=950, plot_height=300, title='Total Count & Average Number of Pages (by Year of Publication)', tools = [hover])
p.quad(top = year_count['Count'],
       bottom = 0, 
       right=year_count['Year']-0.4, 
       left=year_count['Year']+0.4, color = '#7ea4b3',
       legend = 'Number of Books')

# p.yaxis.visible = False
# p.extra_y_ranges = {"NumberOfPages": Range1d(start=0, end=300)}
# p.add_layout(LinearAxis(y_range_name="NumberOfPages"), 'right')
p.line(year_pages['Year'],
       year_pages['NumberOfPages']/40, 
       line_width=2, 
       line_color = '#cfaec6',
       legend = "Average Number of Pages") # dividing # of pages by 40 to fit in plot

plot_aes(p, legend = "top_left")
p.add_layout(citation)
show(p)

In [68]:
# save df as a csv to be able to use it in other notebook
year_pages.to_csv('files/section2_books_per_year.csv', index=False)
year_count.to_csv('files/section2_books_per_year_count.csv', index=False)

## 3- Predicting

In [43]:
# number of reviews vs score & ranking category. 
# filter df with only Number of Reviews, Ranking Category and Score
reviewscount_df = goodreads[['NumberOfReviews','ScoreCat', 'Score']]

In [69]:
# plot the results to see if correlation between books sold and # of books in top 400
# hover = HoverTool(
#         tooltips=[
#             ("Value", "$y{int}")
#         ]
#     )

p = BoxPlot(reviewscount_df, 
            values='NumberOfReviews', 
            label='ScoreCat',
            title="Number of Reviews per Star", 
            ylabel='Number of Reviews',
            xlabel='',
            tools = ['box_zoom', 'reset'],
            color = ['#cfaec6'],
            whisker_color='black',
            legend = None,
#             marker= 'x',
            outliers=False,
            plot_width=950, plot_height=400)

p.left[0].formatter.use_scientific = False
p.xaxis.major_label_orientation = 'horizontal'

plot_aes(p)
show(p)

In [70]:
reviewscount_df.to_csv('files/section3_reviews_count.csv', index=False)

## Moving to Sentiment Analysis for the Reviews & Description Columns

In [45]:
from itertools import *
from nltk.tokenize import wordpunct_tokenize 
from __future__ import division

from afinn import Afinn
afinn = Afinn()

Using the AFINN dictionary to perform a sentiment analysis on the Reviews. The output of the AFINN method is a float variable (the AFINN score) that if larger than zero indicates a positive sentiment and less than zero indicates negative sentiment.

In [46]:
# We are interested in analyzing the sentiment score for the Reviews column.
# The AFINN lexicon provides a positivity score for each word, from -5 (most negative) to 5 (most positive).  

# Filtering df with the relevant columns
filtered_df = goodreads[['ScoreCat','Reviews', 'Score', 'Title']]

In [47]:
# computing the average sentiment score for each review. We already cleaned the Reviews columns by removing 'stop words',
# punctuation and converting all lower cases to upper cases 

unigrams = []
afinn_value = []
ranking_category = []
score = []
for ind, review in islice(filtered_df.iterrows(),388):
    unigrams = ( [i for i in wordpunct_tokenize(review['Reviews'])])
    afinn_value.append(np.mean(list(map(lambda x: afinn.score(str(x.encode('utf-8'))), unigrams))))
    ranking_category.append(review['ScoreCat'])
    score.append(review['Score'])

In [48]:
# joining the 3 lists in a dataframe
afinn_plot = pd.DataFrame()
afinn_plot['ranking_category'] = ranking_category
afinn_plot['senti_value'] = afinn_value
afinn_plot['score'] = score

# plotting the boxplots to analyze possible trends between ranking category and afinn score
p = BoxPlot(afinn_plot, 
            values='senti_value', 
            label='ranking_category',
            title='Sentiment Score by Score Category (using AFINN)',
            ylabel='Average sentiment score',
            xlabel='',
            tools = ['box_zoom', 'reset'],
            color = ['#cfaec6'],
            whisker_color='black',
            legend = None,
#             marker= 'x',
            outliers=False,
            plot_width=950, plot_height=400)

p.left[0].formatter.use_scientific = False
p.xaxis.major_label_orientation = 'horizontal'

plot_aes(p)
show(p)

In [71]:
# save df as a csv to be able to use it in other notebook
afinn_plot.to_csv('files/section3_afinn_plot.csv', index=False)

In [54]:
# adding the category ranking variable
hover = HoverTool(
        tooltips=[
            ("GoodReads Score", "$x{int}"),
            ('Afinn Score',"$y")
        ]
    )


p = Scatter(afinn_plot, 
            x="score", 
            y="senti_value", 
            title="GoodReads Stars vs. Sentiment Score (using AFINN)",
            xlabel='GoodReads score', 
            ylabel='Average sentiment score',
            tools = [hover],
            color=color('ranking_category', palette=['#9ecae1','#2171b5','#c6dbef']),
#             color = 'ranking_category',
            plot_width=950, plot_height=400)

p.ray(x=[0], y=[0], length=0, angle=0, line_width=1.5,color = ['#dba087'])
    
plot_aes(p)
show(p)

# Our sentiment scores are somewhat correlated with goodreads score. The sentiment score does not go below
# -1.5 which makes sense. We would not expect the best ever books to have highly negative sentiment score (i.e -3, 
# -4 or -5). However there’s still a small amount of prediction error - some top 100 books have a negative sentiment score.
# We can question the accuracy of the Afinn score to predict the sucess of a book.

# 1- We will look at the positive and negative words that impact the ranking in the Afinn library to understand why
# 2- We will look at another library to help better predict the score

#### Looking at individual positive and negative words and how they impact a book's ranking

In [55]:
# creating a new df from filtered_df
filtered_df_token = filtered_df[['ScoreCat', 'Reviews', 'Score', 'Title']]

# splitting each word in the Reviews column. We dismiss the proper nouns
filtered_df_token['Reviews'] = filtered_df_token['Reviews'].apply(lambda x: [word for word,pos in pos_tag(x.split()) if pos != 'NNP'])
filtered_df_token

# turning the filtered_df_token dataframe into a one-row-per-word-per-bookreview dataframe. We need to turn it into a dictionary and 
# back to a dataframe
filtered_df_token = pd.concat([pd.DataFrame(filtered_df_token.iloc[i].to_dict()) for i in range(len(filtered_df_token))])

# lemmatizing words
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
filtered_df_token['Reviews'] = filtered_df_token['Reviews'].apply(lambda x: lmtzr.lemmatize(x))

# assigning AFINN score to each word
filtered_df_token['afinn_score'] = [afinn.score(word) for word in filtered_df_token['Reviews']]

print filtered_df_token.tail(10)

        Reviews  Score ScoreCat       Title  afinn_score
6697      never   4.24  4 stars  Skullsworn          0.0
6698       read   4.24  4 stars  Skullsworn          0.0
6699     those,   4.24  4 stars  Skullsworn          0.0
6700      pick,   4.24  4 stars  Skullsworn          0.0
6701       know   4.24  4 stars  Skullsworn          0.0
6702      youre   4.24  4 stars  Skullsworn          0.0
6703  fantastic   4.24  4 stars  Skullsworn          4.0
6704  sometimes   4.24  4 stars  Skullsworn          0.0
6705   violent)   4.24  4 stars  Skullsworn         -3.0
6706   journey.   4.24  4 stars  Skullsworn          0.0


In [56]:
# creating a df where we look at each word and compute the average time it appears in whole the reviews
# and the average goodreads score the books containing it received.
afinn_scatter_df = filtered_df_token.groupby('Reviews').agg({'Score': 'mean', 
                                          'afinn_score': 'first', 
                                          'Title':'count'}).rename(columns={'Title':'Count'}).reset_index()

# only keeping the words that appears at least 1000 times
afinn_scatter_df_1500 = afinn_scatter_df[afinn_scatter_df['Count']>1500]
afinn_scatter_df_1500
# removing some punctuations we still have left
punct = [')',',','-','--','.',':']
afinn_scatter_df_1500 = afinn_scatter_df_1500[~afinn_scatter_df_1500['Reviews'].isin(punct)]

In [74]:
# differentiating dots based on Afinn Score
from bokeh.models import Span, ColumnDataSource, Range1d, LabelSet, Label

# adding vertical line
vline = Span(location=1500, dimension='height', line_color='#dba087', line_width=1.5)

# adding label next to vertical line
citation = Label(x=1250, y=3.756,
                 text='1500', 
                 render_mode='css', 
                 border_line_alpha=1.0, 
                 text_font_size = '8pt', 
                 text_color = '#c59079')


tooltips=[
            ('Word', '@Reviews')
        ]

p = Scatter(afinn_scatter_df_1500, 
            x="Count", 
            y="Score", 
            title="Word Count vs. GoodReads Score",
            xlabel='Word Count', 
            ylabel='Average GoodReads Score',
            tooltips = tooltips,
            color=color('afinn_score', palette=['#f7fbff',
                                                '#deebf7',
                                                '#c6dbef',
                                                '#9ecae1',
                                                '#6baed6',
                                                '#4292c6',
                                                '#2171b5',
                                                '#084594']),
            plot_width=950, plot_height=400)


p.renderers.extend([vline])
p.add_layout(citation)
p.x_range = Range1d(0, 17000)

plot_aes(p, legend ='bottom_right')
show(p)

In [58]:
from bokeh.models import Span, ColumnDataSource, Range1d, LabelSet, Label

tooltips=[
            ('Word', '@Reviews')
        ]

p = Scatter(afinn_scatter_df_1500, 
            x="afinn_score", 
            y="Score", 
            title="Afinn Score vs. GoodReads Stars",
            xlabel='Afinn Score', 
            ylabel='Average GoodReads Score',
            tooltips = tooltips,
            color=color('afinn_score', palette=['#f7fbff','#deebf7','#c6dbef','#9ecae1','#6baed6','#2171b5', '#084594']),
            plot_width=950, plot_height=400, legend=False)

p.line([-3.5,3.5], [3.98,4.2], line_width=1.5, line_color = '#dba087')

plot_aes(p)
show(p)

#### Looking at another library (Vader lexicon) to help better predict the score

In [59]:
# using the Vader Lexicon instead
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [60]:
sentences = [review for review in goodreads['Reviews']]
sid = SentimentIntensityAnalyzer()

In [61]:
# create a list of all the VaderScore from the reviews column
vader_score_list = []
for sentence in sentences:
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        vader_score_list.append('{0}: {1}'.format(k, ss[k]))
        
# turn the list into a dataframe and split into 2 columns
vader_df = pd.DataFrame({'VaderScore': vader_score_list})
vader_df = pd.DataFrame(vader_df['VaderScore'].str.split(':',1).tolist(),columns = ['Desc','Score'])

# The compound score is computed by summing the valence scores of each word in the reviews, adjusted according 
# to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive)
# compound score >= 0.5 : positive sentiment: 
# 0.5 < compound score < 0.5: neutral sentiment
# compound score <= -0.5 : negative sentiment: 
# The pos, neu, and neg scores are ratios for proportions of text that fall in each category

In [62]:
# turning the dataframe from long to wide. Crosstab or pivot don't work for this kind of format. Creating separate lists
# and concatening them
compound = []
neg = []
neu = []
pos = []
for index, row in vader_df.iterrows():
    if row['Desc'] == 'compound':
        compound.append(row['Score'])
    elif row['Desc'] == 'neg':
        neg.append(row['Score'])
    elif row['Desc'] == 'neu':
        neu.append(row['Score'])
    else:
        pos.append(row['Score'])
vader_df_wide = pd.DataFrame(np.vstack((compound, neg, neu, pos)).T, columns = ['compound','neg','neu','pos'])

# checking that the length of the new dataframe vader_df_wide is the same as filtered_df
len(filtered_df) == len(vader_df_wide)

# merging the 2 dataframes
filtered_vader_df = pd.concat([filtered_df,vader_df_wide], axis=1)

# turn the compoung, neg, pos and neu control to integer
filtered_vader_df['compound'] = filtered_vader_df['compound'].apply(lambda x: round(float(x),3))
filtered_vader_df['neg'] = filtered_vader_df['neg'].apply(lambda x: round(float(x),2))
filtered_vader_df['neu'] = filtered_vader_df['neu'].apply(lambda x: round(float(x),2))
filtered_vader_df['pos'] = filtered_vader_df['pos'].apply(lambda x: round(float(x),2))

In [63]:
# join with filtered_df_token to add the afinn score
filtered_df_token_for_merge = filtered_df_token[['Title','afinn_score']].groupby('Title').mean().reset_index()
filtered_df_token_for_merge['afinn_score'] = filtered_df_token_for_merge['afinn_score'].round(2)
filtered_vader_df = pd.merge(filtered_vader_df, filtered_df_token_for_merge, how='left', on='Title')

# renaming score column for clariy purposes
filtered_vader_df = filtered_vader_df.rename(columns={'Score': 'goodreads_score'})

In [64]:
from IPython.core.display import HTML
def css():
    style = open("/Users/clairevignon/.ipython/profile_default/static/custom/custom.css", "r").read()
    return HTML(style)
css()